# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686608


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:02<00:41,  1.42s/it]

Rendering models:  13%|█▎        | 4/31 [00:04<00:33,  1.23s/it]

Rendering models:  19%|█▉        | 6/31 [00:04<00:21,  1.14it/s]

Rendering models:  23%|██▎       | 7/31 [00:07<00:34,  1.45s/it]

...

Rendering models:  71%|███████   | 22/31 [00:08<00:01,  5.78it/s]

Rendering models:  77%|███████▋  | 24/31 [00:08<00:00,  7.34it/s]

Rendering models:  84%|████████▍ | 26/31 [00:08<00:00,  9.06it/s]

Rendering models:  97%|█████████▋| 30/31 [00:08<00:00, 11.33it/s]

AlphaPositif                          0.000481
MissDonkey                            0.000355
not-logged-in-f6fac0763296d5759c01    0.000375
not-logged-in-975ec4b89e5ce0be836b    0.000560
not-logged-in-f5ac928c4dcd4e9cb619    0.000444
Dynamatt                              0.000336
not-logged-in-f7e2260ed4a9517a3e4a    0.877638
djswanso                              0.000330
not-logged-in-c6917d9b19f41e6c2995    0.003148
not-logged-in-c3e2bb63f9ec3bdeea91    0.000293
not-logged-in-9a813f8c400d5fedde5d    0.002920
not-logged-in-8998bdc9c49d01c69c85    0.008519
PuZaw                                 0.000691
not-logged-in-5f30b2bb9b7f9d49da41    0.002593
coyotes54                             0.000356
not-logged-in-92faad50242904ab5ca3    0.001547
AlexYoung35                           0.000576
Planetme                              0.008888
rhondacard                            0.004026
Lavadude                              0.042489
Lavadude                              0.031762
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())